![banner](https://mindspore.cn/static/img/logo_black.74c3909e.png)

#  Lecture 21

## 量子模拟

当我还是学生的时候，“量子模拟”这个词是指使用经典计算机来解决量子问题，例如动力学或基态等。我现在想，这个词现在意味着用量子设备解决量子问题。我们感兴趣的是研究模拟一些“局部”哈密顿函数的量子酉运算问题。

让我们回忆一下薛定谔方程（$\hbar = 1$）：
$$
i \frac{d}{d t}|\psi(t)\rangle=H|\psi(t)\rangle
$$
其中$H$被假定为时间无关的哈密顿量。

你只需要知道，可以将上述等式改写如下：
$$
|\psi(t)\rangle=e^{-i H t}|\psi(t=0)\rangle
$$

我们的量子模拟目标是使用一组（基本）量子门（假设初始状态很容易准备或给定）来准备时间相关状态$|\psi(t)\rangle$。

### 定义误差

给定一个运算符（矩阵）$X$，考虑另一个运算符$Y\equiv X+\Delta_x$。我们写为
$$
X+O(\epsilon) \leftrightarrow\left\|\Delta_{X}\right\|=O(\epsilon)
$$
这里没有指定矩阵范数。

回忆一下指数算子的定义，
$$
e^{-i H t}=I-i H t+\sum_{m=2}^{\infty} \frac{(-i H t)^{m}}{m !}
$$

回想一下，算子范数满足以下三角不等式：
$$
\|X+Y\| \leq\|X\|+\|Y\|
$$

因此，我们可以对算子进行绑定：
$$
\left\|\sum_{m=2}^{\infty} \frac{(-i H t)^{m}}{m !}\right\| \leq(\|H\| t)^{2} \sum_{m=0}^{\infty} \frac{(\|H\| t)^{m}}{(m+2) !}
$$

假设我们限制演化时间，这样
$$
\left\|H\right\|t=1
$$

最后一项可以被限制为
$$
\sum_{m=0}^{\infty} \frac{(\|H\| t)^{m}}{(m+2) !} \leq \sum_{m=0}^{\infty} \frac{1}{(m+2) !}=e-1-1<1
$$
这是因为我们可以考虑$e^x=1+x+\sum^\infty_{m=2}\frac{x^m}{m !}$，并且因此$e=1+1+\sum^\infty_{m=0}\frac{1}{(m+1) !}$。

作为结果，我们得到
$$
\boxed{
e^{-i H t}=I-i H t+O\left(\|H\|^{2} t^{2}\right)
}
$$

### k-local 哈密顿量

让我们考虑一个包含 $n$ 个自旋 1/2 粒子（量子比特）的系统，总哈密顿量是自旋之间相互作用的总和，即，
$$
H=\sum^L_{j=1}H_j
$$
其中$H_j$只作用于最多 $k < n$ 个自旋。

重要的一点是你可以改变 $n$，但保持 $k$ 固定。在这里，我们更多地考虑数学，因此没有强加物理局部性，即$k$-local 并不意味着空间中的局部。一个需要解决的技术问题是，我们需要确保$L$不能是指数的。例如，哈密顿量的项恰好包含 $k$ 体相互作用，则
$$
L \le C^n_k=O(n^k)=poly(n)
$$

### 对易情形

作为一种特殊情况，考虑所有项相互对易的情况，即 $[H_j,H_i]=0$。这是有可能发生的，例如伊辛模型： $H=\sum J_{ij} Z_i Z_j$。

在这种情况下，我们可以像这样分离每一项：
$$
\le e^{-i H t}=e^{-i H_{1} t} e^{-i H_{2} t} \cdots e^{-i H_{L} t}
$$

为确保最终误差在 $\epsilon$ 范围内，可以通过要求每个算子有一个以 $\epsilon / L$ 为范围的误差来实现这一点。

如果我们考虑使用$H$门和$T$门模拟每个基本门，那么根据Solavay-Kitaev 定理，我们需要$O(log^c(L/\epsilon))$个门。因为我们有 $L$ 个这样的项，所以我们需要考虑用$O\left(L\enspace log^c \left(L/ \epsilon\right)\right)$个门。

### 不对易情形

在 $[H_j,H_i]\ne 0$ 的情况下，我们的展开形式不再起作用，即
$$
e^{-i H t} \neq e^{-i H_{1} t} e^{-i H_{2} t} \cdots e^{-i H_{L} t}
$$

当然，当 $t$ 很小时（让我们用 $\Delta t$ 代替），我们可以近似地使
$$
e^{-i H \Delta t} \approx e^{-i H_{1} \Delta t} e^{-i H_{2} \Delta t} \cdots e^{-i H_{L} \Delta t}
$$
但我们想让这个近似值尽可能严格。

首先，我们确实有一个等式如下：
$$
e^{-i H t}=\lim _{n \rightarrow \infty}\left(e^{-i H_{1} t / n} e^{-i H_{2} t / n} \cdots e^{-i H_{L} t / n}\right)^{n}
$$

但是这个等式并不那么实用，因为它需要$n \rightarrow \infty$。而我们对有限数量的项更感兴趣。

---

**Lie-Trotter 乘积公式**

对于一对 Hermitian 算子 $A$ 和 $B$ ，其中 $\left\| A \right\| < s < 1$ 和 $\left\| B \right\| < s < 1$ ，我们有
$$
\boxed{
e^{-i A} e^{-i B}=e^{-i(A+B)}+O\left(s^{2}\right)
}
$$

**证明：**

首先， $e^{-iA}=I-iA+O(s^2)$ 和 $e^{-iB}=I-iB+O(s^2)$ ，现在我们可以同时展开它们：
$$
e^{-i A} e^{-i B}=\left(I-i A+O\left(s^{2}\right)\right)\left(I-i B+O\left(s^{2}\right)\right)=I-i(A+B)+O\left(s^{2}\right)
$$

另一方面，我们可能还需要假设：$\left\|A+B\right\| < \left\|A\right\|+\left\|B\right\|<2s<1$，即可得到
$$
e^{-i(A+B)}=I-i(A+B)+O\left(\left\|(A+B)^{2}\right\|\right)
$$
其中$O\left( \left\|(A+B)^2\right\| \right)=O\left( s^2 \right)$。

---

现在，要将 Lie-Trotter 乘积公式应用于模拟问题，我们需要作出以下限制（假设对于每个项，我们有 $\left\|H_i\right\| \Delta t<s<1/L$）：
$$
\left\|H_{1} \Delta t+H_{2} \Delta t+\ldots+H_{L} \Delta t\right\|<L s<1
$$

我们将使用的策略是将这些项一一组合，使用性质$\left\|A\right\|=\left\|AU\right\|$。

让我们考虑乘式
$$
e^{-i H_{1} \Delta t} e^{-i H_{2} \Delta t} \ldots e^{-i H_{L} \Delta t}=\left(e^{-i\left(H_{1}+H_{2}\right) \Delta t}+O\left(s^{2}\right)\right) e^{-i H_{3} \Delta t} \ldots e^{-i H_{L} \Delta t}
$$

然后，我们有
$$
e^{-i\left(H_{1}+H_{2}\right) \Delta t} e^{-i H_{3} \Delta t} \ldots e^{-i H_{L} \Delta t}+O\left(s^{2}\right)
$$
其中我们用了 $e^{-i\left(H_{1}+H_{2}\right) \Delta t}=I-i\left(H_{1}+H_{2}\right) \Delta t+O\left(\left\|\left(H_{1}+H_{2}\right)^{2}\right\| \Delta t^{2}\right)$ 。

接下来，我们只需重复该过程，即：
$$
e^{-i\left(H_{1}+H_{2}\right) \Delta t} e^{-i H_{3} \Delta t}=e^{-i\left(H_{1}+H_{2}+H_{3}\right) \Delta t}+O\left(2^{2} s^{2}\right)+O\left(s^{2}\right)
$$

最后得到的表达式长这样：
$$
\boxed{
e^{-i H_{1} \Delta t} e^{-i H_{2} \Delta t} \cdots e^{-i H_{L} \Delta t}=e^{-i\left(H_{1}+H_{2}+\ldots H_{L}\right) \Delta t}+O\left(L^{3} S^{2}\right)
}
$$

注意$O\left(L^{3} s^{2}\right)=O\left(s^{2}\right)+O\left(2^{2} s^{2}\right)+O\left(3^{2} s^{2}\right)+\ldots O\left((L-1)^{2} s^{2}\right)$，即
$$
O\left(\left\|\left(H_{1}+H_{2}+\ldots H_{k}\right)^{2}\right\| \Delta t^{2}\right)=O\left(k^{2} s^{2}\right)
$$

最后，为了使 $\Delta t=t/N$ 变小，我们可以使 $N$ 变大，
$$
e^{-i H t}=\left(e^{-i\left(H_{1}+H_{2}+\ldots H_{L}\right) t / N}\right)^{N}
$$

问题是，$N$ 应该有多大？ 这取决于最终误差的要求。如果我们希望总误差在 $O(\epsilon)$ 的范围内，那么对于每一项， $e^{-i(H_1+H_2+\dots H_L)t/N}$ 都必须是 $O(\epsilon/N)$。这意味着
$$
O\left(L^{3} s^{2}\right)=O(\epsilon / N)
$$

让我们假设所有的 $\left\|H_i\right\|<H_{max}$，那么$\left\|H_i\right\|\Delta t < H_{max}t/N$。因此，我们有$L^3 H^2_{max}t^2/N^2=\epsilon / N$，这意味着我们需要取
$$
N=O\left(L^{3} H_{\max }^{2} t^{2} / \epsilon\right)
$$

模拟每个如$e^{-iH_k\Delta t}$的项的成本将与对易情况差不多。